In [ ]:
import json, os
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.decomposition import PCA
from wpca import WPCA
from sklearn.preprocessing import StandardScaler
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.optics import euclidean_distance, pixel2world
from aquabyte.visualize import Visualizer
import random
from scipy.stats import norm
from torch.utils.data import Dataset, DataLoader

pd.set_option('display.max_rows', 500)

In [ ]:
f = '/root/data/temp/results_34ff311a393b0e1ee0c885bb542e5424e51b67de_20190926_waiting_pen_depth_experiments_exp_id_10_bremnes_20190806_20190806.h5'
key = 'table'
df = pd.read_hdf(f, key)


In [ ]:
X, labels = [], []
body_parts = sorted([
    'TAIL_NOTCH',
    'ADIPOSE_FIN',
    'ANAL_FIN',
    'PECTORAL_FIN',
    'PELVIC_FIN',
    'DORSAL_FIN',
    'UPPER_LIP',
    'EYE',
    'UPPER_PRECAUDAL_PIT', 
    'LOWER_PRECAUDAL_PIT',
    'HYPURAL_PLATE'
])

for idx, row in df.iterrows():
    if (row.estimated_biomass_g < 0) or (row.estimated_biomass_g > 10000):
        continue
    data_point = []
    left_keypoints_list = row.keypoints.get('leftCrop')
    right_keypoints_list = row.keypoints.get('rightCrop')
    if left_keypoints_list and right_keypoints_list:
        left_keypoints = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in left_keypoints_list}
        right_keypoints = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in right_keypoints_list}
        for bp in body_parts:
            x, y = left_keypoints[bp][0], left_keypoints[bp][1]
            x_right = right_keypoints[bp][0]
            d = abs(x - x_right)
            data_point.append([x, y, d])
        X.append(data_point)
        labels.append(row.estimated_biomass_g)
        
X, labels = np.array(X) / 4096.0, np.array(labels) / 10000.0
 
 

In [ ]:
class KeypointsDataset(Dataset):
    """Keypoints dataset."""

    def __init__(self, X, labels, transform=None):
        self.X = X
        self.labels = labels

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.labels[idx]

        return torch.from_numpy(x).float(), torch.from_numpy(np.array([y])).float()


In [ ]:
# TODO: Define your network architecture here
import torch
from torch import nn

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(33, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.fc5(x)
        x = self.relu(x)
        x = self.fc6(x)
        x = self.relu(x)
        x = self.output(x)
        return x
        



In [ ]:
from torch.utils.data import DataLoader

dataset = KeypointsDataset(X, labels)
dataloader = DataLoader(dataset, batch_size=25, shuffle=True, num_workers=20)


In [ ]:
network = Network()
epochs = 100000
optimizer = torch.optim.Adagrad(network.parameters(), lr=0.01, lr_decay=0.0001)
criterion = torch.nn.L1Loss()

for epoch in range(epochs):
    running_loss = 0.0
    for i, data_batch in enumerate(dataloader):
        optimizer.zero_grad()
        X_batch, y_batch = data_batch
        y_pred = network(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    loss_for_epoch = running_loss / len(dataloader)
    print('Loss for epoch {}: {}'.format(epoch, loss_for_epoch))





In [ ]:
torch.cuda.is_available()